### DataProcessing Notebook

#### 0. Check if Required Packages are Installed

In [ ]:
!conda list numpy
!conda list rdkit
!conda list pandas
!conda list tqdm
!conda list trimesh
!conda list scipy
!conda list pymesh2
!conda list openbabel
!conda list openpyxl
!conda list autodock-vina
!conda list pathlib

#### 1. Extract and Display Molecular Center Coordinates

In [ ]:
# Features: (1) Extract geometric center coordinates of the molecule; (2) Input file is in SDF format; (3) Output molecular center coordinates
import os
import numpy as np
from rdkit import Chem

print("Libraries imported successfully!")
# Main folder and output file path 
base_path = "/home/cenking/VsCode/Delete/Delete_Running/data/TDFA"
input_filename = "BTB13_Coor.sdf"

input_file = os.path.join(base_path, input_filename)
print(f"\n--- Preparing to analyze file: {input_file} ---")

if not os.path.exists(input_file):
    print(f"❌ Error: File not found, please check the path and filename.")
else:
    supplier = Chem.SDMolSupplier(input_file, removeHs=False)
    mol = supplier[0]

    if mol is None:
        print(f"❌ Error: Unable to load molecule from file.")
    elif mol.GetNumConformers() == 0:
        print(f"❌ Error: Molecule does not contain 3D coordinate information.")
    else:
        print("✅ Molecule loaded successfully.")
        conf = mol.GetConformer(0)
        # Calculate and print the current geometric center coordinates of the molecule
        current_center = np.mean([list(conf.GetAtomPosition(j)) for j in range(mol.GetNumAtoms())], axis=0)
        print(f"\n--- Analysis Results ---")
        print(f"Current geometric center of the molecule (X, Y, Z): {np.round(current_center, 4)}")

#### 2. Modify Molecular Coordinates and Save to New File

In [ ]:
# Features: (1) Translate molecular coordinates to specified position; (2) Input file is in SDF format; (3) Output new SDF file
# Tool 2: Translate molecular coordinates and save ---
import os
import numpy as np
from rdkit import Chem

print("Libraries imported successfully!")
# Main folder, input and output file paths
base_path = "/home/cenking/VsCode/Delete/Delete_Running/data/TDFA"
input_filename = "TDFA3.sdf"
output_filename = "TDFA3_Coor.sdf"
# Target center coordinates (X, Y, Z)
target_center_x = -29.3424
target_center_y = -19.8268
target_center_z = -8.5923

target_center = np.array([target_center_x, target_center_y, target_center_z])
input_file = os.path.join(base_path, input_filename)
output_file = os.path.join(base_path, output_filename)
print(f"\n--- Preparing to translate file: {input_file} ---")

if not os.path.exists(input_file):
    print(f"❌ Error: File not found, please check the path and filename.")
else:
    supplier = Chem.SDMolSupplier(input_file, removeHs=False)
    mol = supplier[0]

    if mol and mol.GetNumConformers() > 0:
        print("✅ Molecule loaded successfully, performing translation operation...")
        conf = mol.GetConformer(0)
        current_center = np.mean([list(conf.GetAtomPosition(j)) for j in range(mol.GetNumAtoms())], axis=0)
        translation_vector = target_center - current_center
        # Move coordinates
        for j in range(mol.GetNumAtoms()):
            original_pos = np.array(list(conf.GetAtomPosition(j)))
            new_pos = original_pos + translation_vector
            conf.SetAtomPosition(j, new_pos)
        print(f"Molecule has been translated to a new position centered at {target_center}.")
        # Save the processed molecule
        print(f"--- Saving to new file: {output_file} ---")
        writer = Chem.SDWriter(output_file)
        writer.write(mol)
        writer.close()
        print(f"✅ Success! Processed file has been saved.")
    else:
        print("❌ Error: Failed to load molecule or molecule does not contain 3D conformation.")

#### 3. Save Mol2 as SDF Format File

In [ ]:
# Features: (1) Convert Mol2 file to SDF format; (2) Input file is in Mol2 format; (3) Output SDF format file
import os
from rdkit import Chem

# File paths
input_mol2_path = "/home/cenking/VsCode/Docking_TX/Bigg/10.mol2"
output_sdf_path = "/home/cenking/VsCode/Docking_TX/Bigg/10.sdf"

def convert_mol2_to_sdf(mol2_file, sdf_file):
    if not os.path.exists(mol2_file):
        print(f"Error: Input file does not exist -> {mol2_file}")
        return
    print(f"Reading MOL2 file: {mol2_file}")
    # Load atoms
    mol = Chem.MolFromMol2File(mol2_file, removeHs=False, sanitize=True)

    if mol is None:
        print("Error: Unable to correctly load molecule from MOL2 file. Please check if the file is corrupted or the format is standard.")
        return
    print("Molecule loaded successfully, writing to SDF file...")

    # Create SDF molecule
    try:
        with Chem.SDWriter(sdf_file) as writer:
            mol.SetProp("_Name", os.path.basename(mol2_file).split('.')[0])
            writer.write(mol)
        print(f"\nSuccess! File has been converted to SDF format and saved to:\n{sdf_file}")
    except Exception as e:
        print(f"Error occurred while writing SDF file: {e}")

if __name__ == "__main__":
    if "path/to/your" in input_mol2_path or "path/to/your" in output_sdf_path:
        print("Please modify input_mol2_path and output_sdf_path values in the script to your own file paths first!")
    else:
        convert_mol2_to_sdf(input_mol2_path, output_sdf_path)

正在读取 MOL2 文件: /home/cenking/VsCode/Docking_TX/Bigg/10.mol2
分子加载成功，正在写入 SDF 文件...

成功！文件已转换为 SDF 格式并保存到:
/home/cenking/VsCode/Docking_TX/Bigg/10.sdf


#### 4. Extract SMILES from SDF Molecules

In [ ]:
# Features: (1) Extract SMILES from all SDF files in the specified folder and save to Excel file; (2) Input files are in SDF format; (3) Output Excel file
import os
import pandas as pd
from rdkit import Chem
from tqdm.notebook import tqdm
from pathlib import Path

def extract_smiles_from_sdf_folder(input_folder_path: str, output_excel_path: str):
    input_folder = Path(input_folder_path)
    output_file = Path(output_excel_path)
    # Check if input folder exists
    if not input_folder.is_dir():
        print(f"Error: Input folder does not exist -> {input_folder}")
        return
    
    sdf_files = list(input_folder.glob("*.sdf"))
    
    if not sdf_files:
        print(f"Error: No SDF files found in folder {input_folder}")
        return 
    print(f"Found {len(sdf_files)} SDF files")
    # Store all extracted SMILES
    smiles_list = []
    
    for sdf_file in tqdm(sdf_files, desc="Processing SDF files"):
        try:
            supplier = Chem.SDMolSupplier(str(sdf_file))
            mol = next(supplier)
            
            if mol is not None:
                smiles = Chem.MolToSmiles(mol)
                smiles_list.append(smiles)
            else:
                print(f"Warning: Unable to read molecule from file: {sdf_file.name}")
                
        except Exception as e:
            print(f"Error occurred while processing file {sdf_file.name}: {e}")
            continue
            
    print(f"\nSuccessfully extracted {len(smiles_list)} SMILES")
    
    df = pd.DataFrame({"SMILES": smiles_list})
    # Ensure output directory exists
    output_file.parent.mkdir(parents=True, exist_ok=True)
    
    try:
        df.to_excel(output_file, index=False)
        print(f"Successfully saved SMILES to Excel file: {output_file}")
    except Exception as e:
        print(f"Error occurred while saving Excel file: {e}")

# Input and output file paths
input_folder = r"/home/cenking/VsCode/Delete/Delete_Supporting/RawData/BTB11/BTB_Run_11_20251116_015501"
output_excel = r"/home/cenking/VsCode/Delete/Delete_Supporting/RawData/BTB11/BTB_Run_11_20251116_015501.xlsx" 
extract_smiles_from_sdf_folder(input_folder, output_excel)

#### 5. Calculate Similarity and Average Similarity Between SMILES

In [ ]:
# Features: (1) Calculate Tanimoto similarity between SMILES in the table and reference SMILES; (2) Calculate average similarity of all SMILES in the file; (3) Input file is in Excel format; (4) Output Excel file with similarity
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit import RDLogger
from pathlib import Path
import warnings
# Suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def calculate_similarities(excel_path: str, reference_smiles: str):
    excel_file = Path(excel_path)
    
    if not excel_file.is_file():
        print(f"Error: Excel file does not exist -> {excel_file}")
        return
        
    try:
        
        df = pd.read_excel(excel_file)
        
        if "SMILES" not in df.columns:
            print("Error: No 'SMILES' column in the Excel file")
            return
            
        print(f"Successfully read Excel file, containing {len(df)} SMILES")
        
        ref_mol = Chem.MolFromSmiles(reference_smiles)
        if ref_mol is None:
            print("Error: Unable to parse reference SMILES")
            return
        ref_fp = AllChem.GetMorganFingerprintAsBitVect(ref_mol, 2, 1024)
        
        similarities = []
        valid_mols = []
        valid_fps = []
        
        print("Calculating similarities...")
        for smiles in df['SMILES']:
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024)
                    sim = DataStructs.TanimotoSimilarity(ref_fp, fp)
                    similarities.append(sim)
                    valid_mols.append(mol)
                    valid_fps.append(fp)
                else:
                    similarities.append(None)
            except Exception as e:
                print(f"Error occurred while processing SMILES: {smiles}, Error: {e}")
                similarities.append(None)
                
        df['Tanimoto-Similarity'] = similarities
        
        # Calculate average similarity
        n = len(valid_fps)
        if n > 1:
            total_sim = 0
            pair_count = 0
            for i in range(n):
                for j in range(i+1, n):
                    sim = DataStructs.TanimotoSimilarity(valid_fps[i], valid_fps[j])
                    total_sim += sim
                    pair_count += 1
            avg_internal_sim = total_sim / pair_count
            df['Internal-Average-Similarity'] = avg_internal_sim
            
            print(f"\nAverage Tanimoto similarity between SMILES in the file: {avg_internal_sim:.4f}")
        else:
            print("\nNot enough valid SMILES in the file to calculate internal similarity")
            df['Internal-Average-Similarity'] = None
        
        # Save results
        output_path = excel_file.parent / f"similarity_results_{excel_file.stem}.xlsx"
        df.to_excel(output_path, index=False)
        print(f"\nResults saved to: {output_path}")
        
    except Exception as e:
        print(f"Error occurred during processing: {e}")

# Input file path
excel_path = r"/home/cenking/VsCode/Delete/Delete_Supporting/CRNB_Batch1/Results_with_ni.xlsx"
# Reference SMILES
reference_smiles = r"C=CC(=O)CC(=O)C=C"  
calculate_similarities(excel_path, reference_smiles)

#### 6. Calculate Molecular Weight of SMILES

In [ ]:
# Features: (1) Calculate molecular weight for each SMILES in the table; (2) Calculate average molecular weight; (3) Input file is in Excel format; (4) Output Excel file with molecular weights
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import RDLogger
from pathlib import Path
import warnings
# Completely suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def calculate_molecular_weights(excel_path: str):
    excel_file = Path(excel_path)
    
    if not excel_file.is_file():
        print(f"Error: Excel file does not exist -> {excel_file}")
        return
        
    try:
        df = pd.read_excel(excel_file)
        
        if "SMILES" not in df.columns:
            print("Error: No 'SMILES' column in the Excel file")
            return
            
        print(f"Successfully read Excel file, containing {len(df)} SMILES")

        molecular_weights = []
        valid_weights = []
        
        print("Calculating molecular weights...")
        for smiles in df['SMILES']:
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    mw = Descriptors.MolWt(mol)
                    molecular_weights.append(mw)
                    valid_weights.append(mw)
                else:
                    molecular_weights.append(None)
            except Exception as e:
                print(f"Error occurred while processing SMILES: {smiles}, Error: {e}")
                molecular_weights.append(None)
                
        df['Molecular-Weight'] = molecular_weights
        
        if valid_weights:
            avg_mw = sum(valid_weights) / len(valid_weights)
            df['Average-Molecular-Weight'] = avg_mw
            
            print(f"\nAverage molecular weight: {avg_mw:.2f} g/mol")
            print(f"Successfully calculated molecular weights for {len(valid_weights)} molecules")
        else:
            print("\nNo valid SMILES in the file to calculate molecular weight")
            df['Average-Molecular-Weight'] = None
        
        output_path = excel_file.parent / f"molecular_weight_results_{excel_file.stem}.xlsx"
        df.to_excel(output_path, index=False)
        print(f"\nResults saved to: {output_path}")
        
    except Exception as e:
        print(f"Error occurred during processing: {e}")

# Input file path
excel_path = r"/home/cenking/VsCode/Proby/Proby_Supporting/PostData/NIR_II_Molecule_Generating_by_Reinvent4/NIR_II_Molecule_Generating_by_Reinvent4_Merged_Positive_DMSO_SMILES_Only.xlsx"  
calculate_molecular_weights(excel_path)

#### 7. Calculate Chemical Feasibility Score of SMILES

In [ ]:
# Features: (1) Calculate chemical feasibility score for each SMILES in the table; (2) Score based on: unfavorable rings, unreasonable chemical bonds, Lipinski rule violations, etc.; (3) Input and output files are in Excel format
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors
from rdkit import RDLogger
from pathlib import Path
import warnings
# Suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def calculate_chemical_feasibility(excel_path: str):
    # Convert to path object
    excel_file = Path(excel_path)
    
    # Check if file exists
    if not excel_file.is_file():
        print(f"Error: Excel file does not exist -> {excel_file}")
        return
        
    try:
        # Read Excel file
        df = pd.read_excel(excel_file)
        
        # Check if SMILES column exists
        if "SMILES" not in df.columns:
            print("Error: No 'SMILES' column in the Excel file")
            return
            
        print(f"Successfully read Excel file, containing {len(df)} SMILES")
        
        # Store various metrics
        feasibility_scores = []
        small_rings = []
        unstable_bonds = []
        fused_aromatic_rings = []
        valid_scores = []
        
        print("Calculating chemical feasibility...")
        for smiles in df['SMILES']:
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    penalty = 0  # Penalty score, higher means less feasible
                    
                    # 1. Check small rings (3-membered, 4-membered rings)
                    ring_info = mol.GetRingInfo()
                    small_ring_count = 0
                    for ring in ring_info.AtomRings():
                        ring_size = len(ring)
                        if ring_size == 3:
                            small_ring_count += 1
                            penalty += 20  # 3-membered ring has heavier penalty
                        elif ring_size == 4:
                            small_ring_count += 1
                            penalty += 10  # 4-membered ring has lighter penalty
                    
                    # 2. Check unstable/uncommon chemical bonds
                    unstable_bond_count = 0
                    for bond in mol.GetBonds():
                        begin_atom = bond.GetBeginAtom()
                        end_atom = bond.GetEndAtom()
                        
                        # Check unstable bond combinations
                        # O-O bond
                        if begin_atom.GetSymbol() == 'O' and end_atom.GetSymbol() == 'O':
                            if bond.GetBondType() == Chem.BondType.SINGLE:
                                unstable_bond_count += 1
                                penalty += 5
                        
                        # N-N bond (except for specific cases)
                        if begin_atom.GetSymbol() == 'N' and end_atom.GetSymbol() == 'N':
                            if bond.GetBondType() != Chem.BondType.DOUBLE:
                                unstable_bond_count += 1
                                penalty += 5
                        
                        # Check overcrowded carbon (more than 4 bonds)
                        if begin_atom.GetSymbol() == 'C' and begin_atom.GetDegree() > 4:
                            unstable_bond_count += 1
                            penalty += 10
                        if end_atom.GetSymbol() == 'C' and end_atom.GetDegree() > 4:
                            unstable_bond_count += 1
                            penalty += 10
                    
                    # 3. Check four or more fused aromatic rings
                    fused_aromatic_count = 0
                    aromatic_ring_systems = []
                    
                    # Get all aromatic rings
                    for ring in ring_info.AtomRings():
                        is_aromatic = all(mol.GetAtomWithIdx(atom_idx).GetIsAromatic() for atom_idx in ring)
                        if is_aromatic:
                            aromatic_ring_systems.append(set(ring))
                    
                    # Check if there are 4 or more aromatic rings sharing atoms (fused rings)
                    if len(aromatic_ring_systems) >= 4:
                        # Build fused ring systems
                        ring_clusters = []
                        for ring in aromatic_ring_systems:
                            merged = False
                            for cluster in ring_clusters:
                                # If shares atoms with existing cluster, merge
                                if cluster & ring:
                                    cluster.update(ring)
                                    merged = True
                                    break
                            if not merged:
                                ring_clusters.append(ring.copy())
                        
                        # Check how many rings in each cluster
                        for cluster in ring_clusters:
                            cluster_ring_count = 0
                            for ring in aromatic_ring_systems:
                                if ring & cluster:
                                    cluster_ring_count += 1
                            
                            if cluster_ring_count >= 4:
                                fused_aromatic_count += 1
                                penalty += 15  # Penalty for 4 or more fused aromatic rings
                    
                    # Calculate final feasibility score (100-point scale, higher is better)
                    feasibility_score = max(0, 100 - penalty)
                    
                    feasibility_scores.append(feasibility_score)
                    small_rings.append(small_ring_count)
                    unstable_bonds.append(unstable_bond_count)
                    fused_aromatic_rings.append(fused_aromatic_count)
                    valid_scores.append(feasibility_score)
                    
                else:
                    feasibility_scores.append(None)
                    small_rings.append(None)
                    unstable_bonds.append(None)
                    fused_aromatic_rings.append(None)
                    
            except Exception as e:
                print(f"Error occurred while processing SMILES: {smiles}, Error: {e}")
                feasibility_scores.append(None)
                small_rings.append(None)
                unstable_bonds.append(None)
                fused_aromatic_rings.append(None)
        
        # Add columns
        df['Small-Rings'] = small_rings
        df['Unstable-Bonds'] = unstable_bonds
        df['Fused-Aromatic-Rings'] = fused_aromatic_rings
        df['Feasibility-Score'] = feasibility_scores
        
        # Calculate average feasibility score
        if valid_scores:
            avg_feasibility = sum(valid_scores) / len(valid_scores)
            df['Average-Feasibility-Score'] = avg_feasibility
            
            print(f"\nAverage chemical feasibility score: {avg_feasibility:.2f}")
            print(f"Successfully calculated feasibility scores for {len(valid_scores)} molecules")
        else:
            print("\nNo valid SMILES in the file to calculate feasibility score")
            df['Average-Feasibility-Score'] = None
        
        # Save results
        output_path = excel_file.parent / f"feasibility_results_{excel_file.stem}.xlsx"
        df.to_excel(output_path, index=False)
        print(f"\nResults saved to: {output_path}")
        
    except Exception as e:
        print(f"Error occurred during processing: {e}")

# Input file path
excel_path = r"/home/cenking/VsCode/Proby/Proby_Supporting/PostData/NIR_II_Molecule_Generating_by_Reinvent4/NIR_II_Molecule_Generating_by_Reinvent4_Merged_Positive_DMSO_SMILES_Only.xlsx"  
calculate_chemical_feasibility(excel_path)

#### 8. Batch Docking Scoring with Vina

In [ ]:
# Features: (1) Use Vina for batch docking of all molecules in an SDF file; (2) Output CSV file with docking scores; (3) Save complex files in PDBQT format
import pandas as pd
import os
import subprocess
import csv
import shutil
import tempfile
import warnings
from pathlib import Path
from rdkit import Chem
from rdkit import RDLogger

# Completely suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def vina_batch_docking_pdbqt_only(sdf_file: str, receptor_pdbqt: str, 
                        center_x: float, center_y: float, center_z: float,
                        size_x: float = 20.0, size_y: float = 20.0, size_z: float = 20.0,
                        output_base_path: str = None):
    
    # --- Path setup ---
    sdf_path = Path(sdf_file)
    receptor_path = Path(receptor_pdbqt)
    
    if output_base_path is None:
        base_dir = sdf_path.parent
        stem = sdf_path.stem
    else:
        base_path_obj = Path(output_base_path)
        base_dir = base_path_obj.parent
        stem = base_path_obj.stem

    output_csv = base_dir / f"{stem}_results_pdbqt.csv"
    # Folder name with pdbqt suffix
    complex_dir = base_dir / f"{stem}_complexes_pdbqt"
    complex_dir.mkdir(parents=True, exist_ok=True)

    if not sdf_path.is_file():
        print(f"Error: SDF file does not exist -> {sdf_path}")
        return
    if not receptor_path.is_file():
        print(f"Error: Receptor PDBQT file does not exist -> {receptor_path}")
        return

    print(f"Reading SDF file: {sdf_path}")
    print(f"Results will be written in real-time to: {output_csv}")
    print(f"PDBQT format complexes will be saved in: {complex_dir}")

    # Pre-read receptor PDBQT content to avoid repeated IO in loop
    try:
        with open(receptor_path, 'r') as f:
            receptor_content = f.read()
    except Exception as e:
        print(f"Error: Unable to read receptor PDBQT file. {e}")
        return

    # --- Initialize CSV file ---
    headers = ['Molecule_ID', 'SMILES', 'Vina_Score', 'Complex_File']
    with open(output_csv, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(headers)

    supplier = Chem.SDMolSupplier(str(sdf_path), removeHs=False, sanitize=True)
    temp_root = Path(tempfile.mkdtemp())
    
    mol_count = 0
    success_count = 0

    print("\nStarting streaming batch docking (outputting PDBQT complexes)...")

    for mol in supplier:
        mol_count += 1
        mol_id = f"mol_{mol_count}"
        
        mol_temp_dir = temp_root / mol_id
        mol_temp_dir.mkdir(exist_ok=True)
        
        if mol is None:
            print(f"[{mol_count}] Warning: Unable to read molecule, skipping")
            _write_result(output_csv, [mol_id, "Read_Error", None, None])
            continue
            
        try:
            smiles = Chem.MolToSmiles(Chem.RemoveHs(mol))
            
            if mol.GetNumConformers() == 0:
                print(f"[{mol_count}] Warning: No 3D coordinates, skipping")
                _write_result(output_csv, [mol_id, smiles, None, None])
                continue

            # 1. Prepare ligand PDB -> PDBQT (input for Vina)
            ligand_pdb = mol_temp_dir / "ligand.pdb"
            ligand_pdbqt = mol_temp_dir / "ligand.pdbqt"
            Chem.MolToPDBFile(mol, str(ligand_pdb))

            proc = subprocess.run([
                'obabel', str(ligand_pdb), '-O', str(ligand_pdbqt), '-h'
            ], capture_output=True, text=True, timeout=30)
            
            if proc.returncode != 0 or not ligand_pdbqt.exists():
                print(f"[{mol_count}] Obabel conversion failed")
                _write_result(output_csv, [mol_id, smiles, None, None])
                continue

            # 2. Run Vina docking
            output_vina_pdbqt = mol_temp_dir / "vina_out.pdbqt"
            vina_cmd = [
                'vina',
                '--receptor', str(receptor_path),
                '--ligand', str(ligand_pdbqt),
                '--center_x', str(center_x), '--center_y', str(center_y), '--center_z', str(center_z),
                '--size_x', str(size_x), '--size_y', str(size_y), '--size_z', str(size_z),
                '--out', str(output_vina_pdbqt),
                '--cpu', '4' 
            ]
            
            vina_res = subprocess.run(vina_cmd, capture_output=True, text=True)
            
            best_score = None
            best_model_lines = []
            
            if vina_res.returncode == 0 and output_vina_pdbqt.exists():
                # 3. Parse PDBQT output (extract score and best conformation text)
                with open(output_vina_pdbqt, 'r') as f_v:
                    capture_model = False
                    found_model_1 = False
                    
                    for line in f_v:
                        # Extract score
                        if line.strip().startswith('REMARK VINA RESULT'):
                            parts = line.split()
                            if len(parts) >= 4 and best_score is None:
                                best_score = float(parts[3])

                        # Extract Model 1 text content
                        if line.startswith('MODEL 1'):
                            capture_model = True
                            found_model_1 = True
                        
                        if capture_model:
                            best_model_lines.append(line)
                            # Stop capturing when encountering ENDMDL
                            if line.startswith('ENDMDL'):
                                capture_model = False
                                break 
                
                if best_score is not None and found_model_1:
                    print(f"[{mol_count}] Docking successful: {best_score:.2f} kcal/mol")
                    success_count += 1
                    
                    # 4. Generate complex (plain text merge)
                    # Format: receptor content + ligand best conformation content
                    complex_filename = f"complex_{mol_count}_{best_score:.2f}.pdbqt"
                    complex_path = complex_dir / complex_filename
                    
                    with open(complex_path, 'w') as f_out:
                        f_out.write(receptor_content)
                        f_out.write("\n") # Ensure separation
                        f_out.writelines(best_model_lines)
                    
                    # 5. Write result to CSV
                    _write_result(output_csv, [mol_id, smiles, best_score, complex_filename])
                else:
                    print(f"[{mol_count}] No valid score or Model 1 found")
                    _write_result(output_csv, [mol_id, smiles, None, None])
            else:
                print(f"[{mol_count}] Vina run failed")
                _write_result(output_csv, [mol_id, smiles, None, None])

        except Exception as e:
            print(f"[{mol_count}] Processing exception: {e}")
            _write_result(output_csv, [mol_id, "Error", None, None])
        
        try:
            shutil.rmtree(mol_temp_dir)
        except:
            pass

    try:
        shutil.rmtree(temp_root)
    except:
        pass
        
    print(f"\nTask completed!")
    print(f"Total processed: {mol_count}")
    print(f"Successfully docked: {success_count}")
    print(f"CSV results path: {output_csv}")
    print(f"PDBQT complex folder: {complex_dir}")

def _write_result(csv_path, row_data):
    try:
        with open(csv_path, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow(row_data)
    except Exception as e:
        print(f"Failed to write CSV: {e}")

# --- Parameter configuration ---
sdf_file = r"/home/cenking/VsCode/L1900-Targetmol-Anti-diabetic Compound Library-690cpds_standardized_3d.sdf"
receptor_pdbqt = r"/home/cenking/VsCode/7P9N.pdbqt"
# Output path
output_base = r"/home/cenking/VsCode/docking_results/results_complexes_pdbqt" 

# Docking parameters
center_x = 27.028
center_y = 24.664
center_z = 27.651
size_x = 40.0
size_y = 40.0
size_z = 40.0

# Run
vina_batch_docking_pdbqt_only(sdf_file, receptor_pdbqt, center_x, center_y, center_z, size_x, size_y, size_z, output_base)

#### 9. Calculate Structural Diversity Score of SMILES

In [ ]:
# Features: (1) Score based on: element types, novel atom count, bond types, and novel bond count; (2) Input and output files are in Excel format
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit import RDLogger
from pathlib import Path
import warnings
from collections import Counter
# Suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def calculate_diversity_scores(excel_path: str):
    excel_file = Path(excel_path)
    
    if not excel_file.is_file():
        print(f"Error: Excel file does not exist -> {excel_file}")
        return
        
    common_elements = {'C', 'H', 'N', 'O', 'S', 'P', 'F', 'Cl', 'Br', 'I'}
    common_bond_types = {Chem.BondType.SINGLE, Chem.BondType.DOUBLE, Chem.BondType.TRIPLE, Chem.BondType.AROMATIC}
    
    try:
        df = pd.read_excel(excel_file)
        
        if "SMILES" not in df.columns:
            print("Error: No 'SMILES' column in the Excel file")
            return
            
        print(f"Successfully read Excel file, containing {len(df)} SMILES")
        
        diversity_scores = []
        element_varieties = []
        novel_atom_counts = []
        bond_varieties = []
        novel_bond_counts = []
        valid_scores = []
        
        print("Calculating structural diversity...")
        for smiles in df['SMILES']:
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    # Element type statistics
                    elements = set()
                    novel_atoms = 0
                    for atom in mol.GetAtoms():
                        symbol = atom.GetSymbol()
                        elements.add(symbol)
                        if symbol not in common_elements:
                            novel_atoms += 1
                    
                    element_variety = len(elements)
                    # Bond type statistics
                    bond_types = set()
                    novel_bonds = 0
                    for bond in mol.GetBonds():
                        bond_type = bond.GetBondType()
                        bond_types.add(bond_type)
                        if bond_type not in common_bond_types:
                            novel_bonds += 1
                    
                    bond_variety = len(bond_types)
                    # Calculate diversity score
                    # Note: Scoring formula: weighted combination of various metrics
                    diversity_score = (
                        element_variety * 10 +  
                        novel_atoms * 5 +       
                        bond_variety * 8 +       
                        novel_bonds * 3          
                    )
                    # Normalize to 0-100
                    diversity_score = min(diversity_score, 100)
                    diversity_scores.append(diversity_score)
                    element_varieties.append(element_variety)
                    novel_atom_counts.append(novel_atoms)
                    bond_varieties.append(bond_variety)
                    novel_bond_counts.append(novel_bonds)
                    valid_scores.append(diversity_score)
                    
                else:
                    diversity_scores.append(None)
                    element_varieties.append(None)
                    novel_atom_counts.append(None)
                    bond_varieties.append(None)
                    novel_bond_counts.append(None)
                    
            except Exception as e:
                print(f"Error occurred while processing SMILES: {smiles}, Error: {e}")
                diversity_scores.append(None)
                element_varieties.append(None)
                novel_atom_counts.append(None)
                bond_varieties.append(None)
                novel_bond_counts.append(None)

        df['Element-Variety'] = element_varieties
        df['Novel-Atoms'] = novel_atom_counts
        df['Bond-Variety'] = bond_varieties
        df['Novel-Bonds'] = novel_bond_counts
        df['Diversity-Score'] = diversity_scores
        # Calculate average diversity score
        if valid_scores:
            avg_diversity = sum(valid_scores) / len(valid_scores)
            df['Average-Diversity-Score'] = avg_diversity
            
            print(f"\nAverage structural diversity score: {avg_diversity:.2f}")
            print(f"Successfully calculated diversity scores for {len(valid_scores)} molecules")
        else:
            print("\nNo valid SMILES in the file to calculate diversity score")
            df['Average-Diversity-Score'] = None

        output_path = excel_file.parent / f"diversity_results_{excel_file.stem}.xlsx"
        df.to_excel(output_path, index=False)
        print(f"\nResults saved to: {output_path}")
        
    except Exception as e:
        print(f"Error occurred during processing: {e}")
# Output file directory
excel_path = r"/home/cenking/VsCode/Delete/outputs/Frag1_2/Test_element_0.6.xlsx"  

calculate_diversity_scores(excel_path)

#### 10. Calculate Collision Volume and Percentage Between Molecule and Protein Pocket

In [ ]:
# Features: (1) Comprehensive evaluation script - combining shape similarity and pocket collision analysis; (2) Support batch processing; (3) Output in Excel and PKL formats
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
from pathlib import Path
import warnings
import logging
import traceback
import pickle
from tqdm.notebook import tqdm
from datetime import datetime

# Suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

# Set up logging system
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Check if required packages are installed
try:
    import trimesh
    from scipy.spatial import ConvexHull
except ImportError:
    print("Warning: trimesh and scipy libraries need to be installed")
    print("Please run: pip install trimesh scipy")

def calculate_molecular_volume(mol):
    if mol.GetNumConformers() == 0:
        return None
    
    conf = mol.GetConformer()
    total_volume = 0
    vdw_radii = {
        'H': 1.20, 'C': 1.70, 'N': 1.55, 'O': 1.52, 'F': 1.47,
        'P': 1.80, 'S': 1.80, 'Cl': 1.75, 'Br': 1.85, 'I': 1.98
    }
    
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        radius = vdw_radii.get(symbol, 1.70) 
        volume = (4/3) * np.pi * (radius ** 3)
        total_volume += volume
    
    return total_volume

def calculate_shape_similarity(mol_a, mol_b):
    try:
        return 1 - AllChem.ShapeTanimotoDist(mol_a, mol_b)
    except Exception as e:
        logger.error(f"Error calculating shape similarity: {str(e)}")
        return None

def point_in_mesh(point, mesh):
    try:
        return mesh.contains([point])[0]
    except:
        return False

def calculate_pocket_overlap_volume(mol, pocket_mesh):
    """Calculate collision volume and percentage between molecule and pocket"""
    if mol.GetNumConformers() == 0:
        return None, None, None
    
    conf = mol.GetConformer()
    mol_volume = calculate_molecular_volume(mol)
    
    if mol_volume is None:
        return None, None, None
    
    vdw_radii = {
        'H': 1.20, 'C': 1.70, 'N': 1.55, 'O': 1.52, 'F': 1.47,
        'P': 1.80, 'S': 1.80, 'Cl': 1.75, 'Br': 1.85, 'I': 1.98
    }
    
    overlap_volume = 0
    
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        point = np.array([pos.x, pos.y, pos.z])
        
        if point_in_mesh(point, pocket_mesh):
            symbol = atom.GetSymbol()
            radius = vdw_radii.get(symbol, 1.70)
            atom_volume = (4/3) * np.pi * (radius ** 3)
            overlap_volume += atom_volume
    
    overlap_percentage = (overlap_volume / mol_volume) * 100 if mol_volume > 0 else 0
    
    return mol_volume, overlap_volume, overlap_percentage

def extract_vina_affinity(mol):
    try:
        if mol.HasProp('REMARK'):
            remark = mol.GetProp('REMARK')
            for line in remark.splitlines():
                if 'VINA RESULT' in line or line.strip().startswith('1'):
                    parts = line.split()
                    if len(parts) >= 2:
                        return float(parts[1])
        return None
    except Exception as e:
        logger.debug(f"Error extracting Vina affinity: {str(e)}")
        return None

def comprehensive_molecular_evaluation(
    sdf_file: str, 
    reference_sdf: str = None,
    pocket_ply: str = None,
    output_excel: str = None,
    output_pkl: str = None,
    calculate_shape: bool = True,
    calculate_pocket: bool = True
):
    sdf_path = Path(sdf_file)
    
    if not sdf_path.is_file():
        logger.error(f"Error: SDF file does not exist -> {sdf_path}")
        return
    
    # Load reference molecule
    reference_mol = None
    if calculate_shape and reference_sdf:
        ref_path = Path(reference_sdf)
        if ref_path.is_file():
            try:
                ref_supplier = Chem.SDMolSupplier(str(ref_path), removeHs=False, sanitize=True)
                reference_mol = next((mol for mol in ref_supplier if mol is not None), None)
                if reference_mol:
                    logger.info(f"✅ Reference molecule loaded successfully")
                else:
                    logger.warning(f"Warning: Unable to load molecule from reference file")
            except Exception as e:
                logger.error(f"Error loading reference molecule: {str(e)}")
        else:
            logger.warning(f"Warning: Reference SDF file does not exist -> {ref_path}")
    
    # Load pocket mesh
    pocket_mesh = None
    if calculate_pocket and pocket_ply:
        ply_path = Path(pocket_ply)
        if ply_path.is_file():
            try:
                pocket_mesh = trimesh.load(str(ply_path))
                logger.info(f"✅ Pocket mesh loaded successfully")
            except Exception as e:
                logger.error(f"Error loading pocket mesh: {str(e)}")
        else:
            logger.warning(f"Warning: PLY file does not exist -> {ply_path}")
    
    try:
        logger.info(f"Reading SDF file: {sdf_path}")
        supplier = Chem.SDMolSupplier(str(sdf_path), removeHs=False, sanitize=True)
        
        # Store results
        results = []
        
        mol_count = 0
        logger.info("\nStarting comprehensive evaluation...")
        
        for mol in tqdm(supplier, desc="Processing molecules"):
            if mol is None:
                logger.warning(f"Warning: Unable to read molecule {mol_count + 1}, skipping")
                continue
                
            mol_count += 1
            
            try:
                # Basic information
                smiles = Chem.MolToSmiles(Chem.RemoveHs(mol))
                mol_name = mol.GetProp("_Name") if mol.HasProp("_Name") else f"Molecule_{mol_count}"
                
                result = {
                    'Molecule_Name': mol_name,
                    'SMILES': smiles,
                    'Num_Atoms': mol.GetNumAtoms()
                }
                
                # Check 3D coordinates
                if mol.GetNumConformers() == 0:
                    logger.warning(f"Warning: Molecule {mol_count} has no 3D coordinates, skipping partial evaluation")
                    result['Has_3D_Coords'] = False
                else:
                    result['Has_3D_Coords'] = True
                    
                    # 1. Shape similarity evaluation
                    if calculate_shape and reference_mol:
                        shape_score = calculate_shape_similarity(mol, reference_mol)
                        result['Shape_Similarity'] = shape_score
                    
                    # 2. Pocket collision evaluation
                    if calculate_pocket and pocket_mesh:
                        mol_vol, overlap_vol, overlap_pct = calculate_pocket_overlap_volume(mol, pocket_mesh)
                        result['Molecular_Volume'] = mol_vol
                        result['Collision_Volume'] = overlap_vol
                        result['Overlap_Percentage'] = overlap_pct
                    
                    # 3. Vina affinity (if exists)
                    vina_affinity = extract_vina_affinity(mol)
                    if vina_affinity is not None:
                        result['Vina_Affinity'] = vina_affinity
                
                # Save RDKit molecule object (for PKL)
                result['rdmol'] = mol
                
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error processing molecule {mol_count}: {str(e)}")
                logger.debug(traceback.format_exc())
                continue
        
        logger.info(f"\n✅ Successfully processed {len(results)} molecules")
        
        # Save as Excel
        if output_excel:
            df_results = []
            for r in results:
                df_row = {k: v for k, v in r.items() if k != 'rdmol'}
                df_results.append(df_row)
            
            df = pd.DataFrame(df_results)
            
            # Calculate averages
            if calculate_shape and 'Shape_Similarity' in df.columns:
                valid_shape = df['Shape_Similarity'].dropna()
                if len(valid_shape) > 0:
                    df['Average_Shape_Similarity'] = valid_shape.mean()
                    logger.info(f"Average shape similarity: {valid_shape.mean():.4f}")
            
            if calculate_pocket and 'Overlap_Percentage' in df.columns:
                valid_overlap = df['Overlap_Percentage'].dropna()
                if len(valid_overlap) > 0:
                    df['Average_Overlap_Percentage'] = valid_overlap.mean()
                    logger.info(f"Average pocket overlap percentage: {valid_overlap.mean():.2f}%")
            
            if 'Vina_Affinity' in df.columns:
                valid_vina = df['Vina_Affinity'].dropna()
                if len(valid_vina) > 0:
                    df['Average_Vina_Affinity'] = valid_vina.mean()
                    logger.info(f"Average Vina affinity: {valid_vina.mean():.2f}")
            
            output_excel_path = Path(output_excel)
            output_excel_path.parent.mkdir(parents=True, exist_ok=True)
            df.to_excel(output_excel_path, index=False)
            logger.info(f"\n📊 Excel results saved to: {output_excel_path}")
        
        # Save as PKL (including RDKit molecule objects)
        if output_pkl:
            output_pkl_path = Path(output_pkl)
            output_pkl_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Sort by shape similarity (if exists)
            if calculate_shape and reference_mol:
                results_sorted = sorted(
                    [r for r in results if r.get('Shape_Similarity') is not None],
                    key=lambda x: x['Shape_Similarity'],
                    reverse=True
                )
                if results_sorted:
                    results = results_sorted
            
            with open(output_pkl_path, 'wb') as f:
                pickle.dump(results, f)
            logger.info(f"💾 PKL results saved to: {output_pkl_path}")
        
        return results
        
    except Exception as e:
        logger.error(f"Error during processing: {str(e)}")
        logger.debug(traceback.format_exc())
        return None
# Input and output file paths
sdf_file = r"/home/cenking/VsCode/Delete/outputs/Test_Frag_1_2/Origin_Data/Test_Bond_0.1.sdf"
reference_sdf = r"/home/cenking/VsCode/Delete/data/workdir/3cl/3cl_ligand.sdf" 
pocket_ply = r"/home/cenking/VsCode/Delete/outputs/7WMM_Macro_pocket_8.0.ply" 
output_excel = r"/home/cenking/VsCode/Delete/outputs/Test_Frag_1_2/comprehensive_evaluation_results.xlsx"
output_pkl = r"/home/cenking/VsCode/Delete/outputs/Test_Frag_1_2/comprehensive_evaluation_results.pkl"

# Execute comprehensive evaluation
results = comprehensive_molecular_evaluation(
    sdf_file=sdf_file,
    # Set to None if shape similarity is not needed
    reference_sdf=reference_sdf, 
    # Set to None if pocket collision is not needed
    pocket_ply=pocket_ply,        
    output_excel=output_excel,
    output_pkl=output_pkl,
    # Whether to calculate shape similarity
    calculate_shape=True,         
    # Whether to calculate pocket collision
    calculate_pocket=True        
)

logger.info("\n🎉 Comprehensive evaluation completed!")

#### 11. Merge SDF Files in Folder

In [ ]:
# Features: (1) Merge all SDF files in a folder into one large SDF file; (2) Each input SDF file should contain one molecule with 3D coordinates; (3) Output as a single SDF file
import os
from rdkit import Chem
from rdkit import RDLogger
from pathlib import Path
from tqdm.notebook import tqdm
import warnings
# Suppress all RDKit warnings and logs
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

def merge_sdf_files(input_folder: str, output_sdf: str):
    folder_path = Path(input_folder)
    output_path = Path(output_sdf)

    if not folder_path.is_dir():
        print(f"Error: Input folder does not exist -> {folder_path}")
        return
    sdf_files = sorted(list(folder_path.glob("*.sdf")))
    
    if not sdf_files:
        print(f"Error: No SDF files found in folder {folder_path}")
        return
    print(f"Found {len(sdf_files)} SDF files")
    print(f"Input folder: {folder_path}")
    print(f"Output file: {output_path}")
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    writer = Chem.SDWriter(str(output_path))
    
    mol_count = 0
    success_count = 0
    
    print("\nStarting to merge SDF files...")
    
    # Use tqdm to display progress
    for sdf_file in tqdm(sdf_files, desc="Merging SDF files"):
        try:

            supplier = Chem.SDMolSupplier(str(sdf_file), removeHs=False, sanitize=True)
            
            for mol in supplier:
                if mol is not None:
                    mol_count += 1
                    
                    if mol.GetNumConformers() == 0:
                        print(f"Warning: Molecule in file {sdf_file.name} has no 3D coordinates")
                    
                    if not mol.HasProp("_Name") or mol.GetProp("_Name") == "":
                        mol.SetProp("_Name", sdf_file.stem)
                    
                    writer.write(mol)
                    success_count += 1
                else:
                    print(f"Warning: Unable to read molecule from file: {sdf_file.name}")
                    
        except Exception as e:
            print(f"Error processing file {sdf_file.name}: {e}")
            continue
    
    # Close writer
    writer.close()
    
    print(f"\n✅ Merge completed!")
    print(f"Processed {len(sdf_files)} SDF files")
    print(f"Successfully merged {success_count} molecules to output file")
    print(f"Output file saved to: {output_path}")
    
    try:
        output_supplier = Chem.SDMolSupplier(str(output_path), removeHs=False)
        output_mol_count = len([mol for mol in output_supplier if mol is not None])
        print(f"Verification: Output file contains {output_mol_count} molecules")
    except Exception as e:
        print(f"Error verifying output file: {e}")

# Input and output file paths
input_folder = r"/home/cenking/VsCode/Delete/outputs/Test_Frag_1_2/Origin_Data/Test_Bond_0.1"  
output_sdf = r"/home/cenking/VsCode/Delete/outputs/Test_Frag_1_2/Origin_Data/Test_Bond_0.1.sdf"  

merge_sdf_files(input_folder, output_sdf)

#### 12. Merge SMILES from Excel Files in Folder

In [ ]:
# Features: (1) Each input Excel file should contain a SMILES column; (2) Output as a single Excel file; (3) Vertically merge all Excel files in the folder into one large Excel file
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
def merge_smiles_excel_files(input_folder: str, output_excel: str):
    folder_path = Path(input_folder)
    output_path = Path(output_excel)
    # Check if input folder exists
    if not folder_path.is_dir():
        print(f"Error: Input folder does not exist -> {folder_path}")
        return
    # Get all Excel files (.xlsx and .xls)
    excel_files = sorted(list(folder_path.glob("*.xlsx")) + list(folder_path.glob("*.xls")))
    if not excel_files:
        print(f"Error: No Excel files found in folder {folder_path}")
        return
    print(f"Found {len(excel_files)} Excel files")
    print(f"Input folder: {folder_path}")
    print(f"Output file: {output_path}")
    # Store all SMILES
    all_dataframes = []
    total_smiles = 0
    success_count = 0
    print("\nStarting to merge Excel files...")
    
    for excel_file in tqdm(excel_files, desc="Processing Excel files"):
        try:
            df = pd.read_excel(excel_file)
            if 'SMILES' not in df.columns:
                print(f"Warning: File {excel_file.name} does not have 'SMILES' column, skipping")
                continue
            df_smiles = df[['SMILES']].copy()
            # Add source file information
            df_smiles['Source_File'] = excel_file.name
            all_dataframes.append(df_smiles)
            total_smiles += len(df_smiles)
            success_count += 1 
        except Exception as e:
            print(f"Error processing file {excel_file.name}: {e}")
            continue
    if not all_dataframes:
        print("\nError: No Excel files with SMILES column were successfully read")
        return
    
    print("\nMerging data...")
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Remove duplicate SMILES
    original_count = len(merged_df)
    # Ensure output directory exists and save merged file
    output_path.parent.mkdir(parents=True, exist_ok=True)
    merged_df.to_excel(output_path, index=False)
    print(f"\n✅ Merge completed!")
    print(f"Processed {len(excel_files)} Excel files")
    print(f"Successfully read {success_count} files with SMILES column")
    print(f"Total merged {len(merged_df)} SMILES")
    print(f"Output file saved to: {output_path}")

# Input and output file paths
input_folder = r"/home/cenking/VsCode/Delete/outputs/Formal_Frag_2" 
output_excel = r"/home/cenking/VsCode/Delete/outputs/Formal_Frag_2/merged_output.xlsx"  

merge_smiles_excel_files(input_folder, output_excel)

#### 13. Convert 2D SDF File to 3D SDF File

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.notebook import tqdm  # Progress bar specifically for Jupyter Notebook

def convert_sdf_2d_to_3d_absolute(input_path, output_path):
    # --- [Modification 1] Absolute path processing ---
    # Force conversion to absolute path regardless of whether relative path or filename is passed
    abs_input = os.path.abspath(input_path)
    abs_output = os.path.abspath(output_path)
    
    # Check if input file exists
    if not os.path.exists(abs_input):
        print(f"Error: File not found, please check the path:\n{abs_input}")
        return

    print(f"Input file absolute path: {abs_input}")
    print(f"Output file absolute path: {abs_output}")
    print("-" * 30)

    # 1. Read SDF file
    # sanitize=False can sometimes prevent errors when reading non-standard SDF files, but we keep the default here to ensure quality
    suppl = Chem.SDMolSupplier(abs_input)
    
    # Get total number of molecules for progress bar (Note: len() may take a while to scan for very large files)
    total_mols = len(suppl) 
    
    # 2. Create writer for output file
    writer = Chem.SDWriter(abs_output)
    
    success_count = 0
    fail_count = 0

    # --- [Modification 2] tqdm.notebook progress bar ---
    # total=total_mols lets the progress bar know when to end
    # desc is the description text on the left side of the progress bar
    for idx, mol in tqdm(enumerate(suppl), total=total_mols, desc="3D Construction Progress"):
        
        if mol is None:
            # Skip molecules that cannot be read
            continue
            
        try:
            # [Core Step 1] Add hydrogens
            mol_3d = Chem.AddHs(mol)
            
            # [Core Step 2] Generate 3D conformation (ETKDGv3)
            params = AllChem.ETKDGv3()
            params.randomSeed = 0xf00d
            embed_res = AllChem.EmbedMolecule(mol_3d, params)
            
            if embed_res == -1:
                fail_count += 1
                continue
            
            # [Core Step 3] Force field optimization (MMFF94)
            try:
                AllChem.MMFFOptimizeMolecule(mol_3d)
            except Exception:
                pass

            # Write to file
            writer.write(mol_3d)
            success_count += 1
            
        except Exception as e:
            # Catch other unexpected errors
            fail_count += 1

    writer.close()
    
    print("\n" + "="*30)
    print(f"Processing completed!")
    print(f"Success: {success_count} / Total: {total_mols}")
    print(f"File saved to:\n{abs_output}")

# --- Usage Example ---
# Even if you pass a relative path, the function will automatically handle it internally
input_filename = "/home/cenking/VsCode/20250225-L1200-Kinase-Inhibitor-Library.SDF"  
output_filename = "/home/cenking/VsCode/20250225-L1200-Kinase-Inhibitor-Library_3D.SDF"

# Run function
convert_sdf_2d_to_3d_absolute(input_filename, output_filename)

#### 14. Standardize SMILES

In [ ]:
import pandas as pd
from rdkit import Chem
from tqdm.notebook import tqdm
import os

def standardize_smiles_csv(input_path, output_path):
    """
    Use RDKit to standardize SMILES column in CSV file
    
    Parameters:
    input_path: Absolute path of input CSV
    output_path: Absolute path of output CSV
    """
    
    # 1. Absolute path validation and check
    input_path = os.path.abspath(input_path)
    output_path = os.path.abspath(output_path)
    
    if not os.path.exists(input_path):
        print(f"Error: Input file not found {input_path}")
        return

    # 2. Read CSV
    # Assume file has only one column with header 'SMILES'
    df = pd.read_csv(input_path)
    
    if 'SMILES' not in df.columns:
        print(f"Error: 'SMILES' column not found in CSV file. Current column names are: {list(df.columns)}")
        return

    standardized_smiles = []

    # 3. Use tqdm.notebook for progress monitoring
    # Iterate through SMILES column and standardize
    for smi in tqdm(df['SMILES'], desc="Standardizing SMILES"):
        try:
            # RDKit standardization logic
            mol = Chem.MolFromSmiles(str(smi))
            if mol:
                # canonical=True ensures output is unique canonical SMILES
                # isomericSmiles=True preserves stereochemistry information (set to False if not needed)
                std_smi = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=True)
                standardized_smiles.append(std_smi)
            else:
                # If SMILES is invalid, keep original or mark as None
                standardized_smiles.append(None) 
        except Exception:
            standardized_smiles.append(None)

    # 4. Generate results and save
    df['SMILES'] = standardized_smiles
    
    # Optional: Remove rows with standardization failure (None)
    # df = df.dropna(subset=['SMILES'])
    
    df.to_csv(output_path, index=False)
    print(f"Processing completed!\nInput path: {input_path}\nOutput path: {output_path}")

# --- Usage Example ---
# Please replace the paths below with absolute paths on your computer
input_file = r"/home/cenking/VsCode/L1900-Targetmol-Anti-diabetic Compound Library-690cpds.csv" 
output_file = r"/home/cenking/VsCode/L1900-Targetmol-Anti-diabetic Compound Library-690cpds_standardized.csv"

standardize_smiles_csv(input_file, output_file)

#### 15. Remove Specific Molecules from SDF File

In [ ]:
import os
from tqdm.notebook import tqdm

def skip_first_n_molecules_sdf(input_path, output_path, n_skip):
    """
    Remove the first n_skip molecules from an SDF file and save the remaining molecules to a new file.
    Uses absolute paths and displays progress with tqdm.notebook.
    """
    # Get absolute paths
    abs_input_path = os.path.abspath(input_path)
    abs_output_path = os.path.abspath(output_path)
    
    print(f"Input file (absolute path): {abs_input_path}")
    print(f"Output file (absolute path): {abs_output_path}")
    
    if not os.path.exists(abs_input_path):
        print(f"Error: Input file not found: {abs_input_path}")
        return

    mol_count = 0
    saved_count = 0
    buffer = []
    
    # First calculate total number of molecules for tqdm progress bar (optional, can skip this step for very large files to save time)
    # Here we do a quick scan to calculate the total for better interactive experience
    print("Calculating total number of molecules...")
    total_mols = 0
    with open(abs_input_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if line.startswith("$$$$"):
                total_mols += 1
    
    print(f"Total molecules: {total_mols}. Preparing to skip first {n_skip}.")

    with open(abs_input_path, 'r', encoding='utf-8', errors='ignore') as f_in, \
         open(abs_output_path, 'w', encoding='utf-8') as f_out:
        
        pbar = tqdm(total=total_mols, desc="Processing Molecules")
        
        for line in f_in:
            buffer.append(line)
            if line.startswith("$$$$"):
                mol_count += 1
                pbar.update(1)
                
                if mol_count > n_skip:
                    f_out.writelines(buffer)
                    saved_count += 1
                
                buffer = [] # Clear buffer
        
        pbar.close()

    print(f"Processing completed.")
    print(f"Total skipped: {min(n_skip, mol_count)}")
    print(f"Total saved: {saved_count}")
    print(f"New file saved to: {abs_output_path}")

# Usage example
input_sdf = "/home/cenking/VsCode/L1900-Targetmol-Anti-diabetic Compound Library-690cpds_standardized_3d.sdf"
output_sdf = "/home/cenking/VsCode/L1900-Targetmol-Anti-diabetic Compound Library-690cpds_standardized_3d_not_skipped.sdf"
n_to_remove = 100
skip_first_n_molecules_sdf(input_sdf, output_sdf, n_to_remove)

#### 16. Generate 3D SDF File

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.notebook import tqdm

# 1. Define molecule
smiles = "B([C@H]1CCCN1C(=O)[C@@H](C(C)C)N)(O)O"
mol_name = "bortezomib_fragment"
smiles_list = [smiles]

# 2. Set absolute paths (save both SDF and PDB for docking software compatibility)
current_dir = os.getcwd()
sdf_path = os.path.join(current_dir, f"{mol_name}_docking_ready.sdf")
pdb_path = os.path.join(current_dir, f"{mol_name}_docking_ready.pdb")

print(f"Target SDF path: {sdf_path}")
print(f"Target PDB path: {pdb_path}")

writer = Chem.SDWriter(sdf_path)

for smi in tqdm(smiles_list, desc="Generating Docking-Ready Structures"):
    mol = Chem.MolFromSmiles(smi)
    
    if mol:
        # A. Add hydrogens (required for docking)
        mol_h = Chem.AddHs(mol)
        
        # B. Key step: Calculate Gasteiger partial charges
        # Without this step, many docking software will report "No charges found" error
        AllChem.ComputeGasteigerCharges(mol_h)
        
        # C. Use ETKDGv3 algorithm to generate multiple conformations
        # Generate 50 conformations to avoid issues with random conformations due to atom collisions
        params = AllChem.ETKDGv3()
        params.useSmallRingTorsions = True # Important for proline rings
        
        # Generate conformation ID list
        conf_ids = AllChem.EmbedMultipleConfs(mol_h, numConfs=50, params=params)
        
        if conf_ids:
            min_energy = float('inf')
            best_conf_id = -1
            
            # D. Optimize each conformation with UFF (supports Boron) and find the lowest energy one
            for conf_id in conf_ids:
                try:
                    res = AllChem.UFFOptimizeMolecule(mol_h, confId=conf_id)
                    if res == 0: # Optimization converged
                        # Calculate energy as selection criterion
                        ff = AllChem.UFFGetMoleculeForceField(mol_h, confId=conf_id)
                        e = ff.CalcEnergy()
                        if e < min_energy:
                            min_energy = e
                            best_conf_id = conf_id
                except Exception:
                    continue
            
            if best_conf_id != -1:
                # E. Save best conformation
                # Set best conformation coordinates as molecule's current coordinates
                # Write to SDF (includes charge properties)
                writer.write(mol_h, confId=best_conf_id)
                
                # Also write to PDB file (many docking workflows prefer PDB)
                Chem.MolToPDBFile(mol_h, pdb_path, confId=best_conf_id)
                print(f"Success! Best conformation energy: {min_energy:.2f} kcal/mol")
            else:
                print(f"All conformations optimization failed: {smi}")
        else:
            print(f"3D embedding failed (ETKDGv3): {smi}")
    else:
        print(f"SMILES parsing error: {smi}")

writer.close()
print("Processing completed. Please try using the generated .pdb or .sdf file for docking.")

目标 SDF 路径: /home/cenking/VsCode/Delete/Delete_Supporting/bortezomib_fragment_docking_ready.sdf
目标 PDB 路径: /home/cenking/VsCode/Delete/Delete_Supporting/bortezomib_fragment_docking_ready.pdb


生成 Docking 专用结构:   0%|          | 0/1 [00:00<?, ?it/s]

成功！最佳构象能量: 21.67 kcal/mol
处理完成。请尝试使用生成的 .pdb 或 .sdf 文件进行 Docking。
